In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, datasets, optimizers

# 1. Chargement des données (comme avant)
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

# 2. Chargement de la base VGG16 pré-entraînée
# include_top=False : On enlève la partie classification (les 1000 classes d'ImageNet)
# weights='imagenet' : On garde les connaissances acquises sur ImageNet
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# On "gèle" la base pour ne pas détruire les connaissances existantes lors du premier entrainement
base_model_vgg.trainable = False 

# 3. Construction du modèle complet
model_vgg = models.Sequential()
model_vgg.add(base_model_vgg)                 # On ajoute VGG
model_vgg.add(layers.Flatten())               # On aplatit la sortie
model_vgg.add(layers.Dense(256, activation='relu')) # Nouvelle couche dense
model_vgg.add(layers.Dropout(0.5))            # Dropout pour éviter le sur-apprentissage
model_vgg.add(layers.Dense(10))               # Sortie CIFAR-10


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 67s 0us/step


In [2]:
model_vgg.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,586 (56.64 MB)

 Trainable params: 133,898 (523.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [4]:
base_model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [ ]:

# 4. Compilation
model_vgg.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 5. Entraînement (Ce sera plus lent car le modèle est gros)
print("Entraînement avec VGG16...")
model_vgg.fit(train_images, train_labels, epochs=3, validation_data=(test_images, test_labels))

Entraînement avec VGG16...
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 308s 195ms/step - accuracy: 0.4179 - loss: 1.6590 - val_accuracy: 0.5552 - val_loss: 1.2697
Epoch 2/3
1230/1563 ━━━━━━━━━━━━━━━━━━━━ 51s 154ms/step - accuracy: 0.5447 - loss: 1.3020

In [5]:
from tensorflow.keras.applications import InceptionV3

# 1. Chargement de la base InceptionV3
# Notez qu'on ne spécifie pas input_shape ici car on va redimensionner avant
base_model_inc = InceptionV3(weights='imagenet', include_top=False)
base_model_inc.trainable = False

# 2. Construction du modèle avec Redimensionnement
model_inc = models.Sequential()

# --- Astuce Importante ---
# On ajoute une couche pour agrandir l'image x3 (32x3 -> 96x96 pixels)
# Cela permet de satisfaire la condition minimale d'Inception
model_inc.add(layers.UpSampling2D(size=(3, 3), input_shape=(32, 32, 3)))

model_inc.add(base_model_inc)                 # On ajoute Inception
model_inc.add(layers.GlobalAveragePooling2D()) # Plus efficace que Flatten pour Inception
model_inc.add(layers.Dense(256, activation='relu'))
model_inc.add(layers.Dropout(0.5))
model_inc.add(layers.Dense(10))

# 3. Compilation
model_inc.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [6]:
base_model_inc.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                    

In [ ]:
#4. Entraînement
# Attention : C'est TRÈS lourd en calcul car on traite des images 96x96 à travers un gros réseau
print("Entraînement avec InceptionV3 (Attention : lent sur CPU)...")
model_inc.fit(train_images, train_labels, epochs=2, validation_data=(test_images, test_labels))

# Fine Tuning

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers

# Supposons que 'model_vgg' et 'base_model_vgg' existent déjà et ont été entraînés (étape précédente)

# 1. Dégeler le modèle de base
base_model_vgg.trainable = True

# 2. Affiner le dégel : On veut geler les couches 1 à 4, et laisser la 5 active
# On parcourt toutes les couches du modèle de base
set_trainable = False
for layer in base_model_vgg.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# Vérification : Combien de variables sont maintenant entraînables ?
print(f"Nombre de couches dans la base : {len(base_model_vgg.layers)}")
# Seules les couches du block5 + vos couches Dense seront entraînables

# 3. Re-Compilation CRUCIALE
# Il faut utiliser un taux d'apprentissage (learning rate) TRÈS BAS.
# Si le taux est trop haut (ex: 0.001), vous allez "casser" le modèle.
# On utilise généralement 1e-5 (0.00001).
model_vgg.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

# 4. Re-Entraînement (Fine-Tuning)
# On entraîne généralement sur un peu plus d'époques, car l'apprentissage est lent.
print("Début du Fine-Tuning...")
history_fine = model_vgg.fit(train_images, train_labels, 
                             epochs=10,
                             initial_epoch=5, # On recommence à l'époque 5 (si on en avait fait 5 avant)
                             validation_data=(test_images, test_labels))

### Risque principal : L'Overfitting (Sur-apprentissage)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Création du callback
arret_precoce = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Ajout dans le fit
model_vgg.fit(train_images, train_labels, 
              epochs=20, # On peut mettre beaucoup d'époques, il s'arrêtera tout seul
              validation_data=(test_images, test_labels),
              callbacks=[arret_precoce]) # <--- On l'ajoute ici

In [ ]:
import matplotlib.pyplot as plt

def comparer_historiques(history_base, history_fine, initial_epochs=5):
    """
    Trace les courbes de précision et de perte en combinant deux historiques.
    """
    
    # 1. Concaténation des données de précision (Accuracy)
    acc = history_base.history['accuracy'] + history_fine.history['accuracy']
    val_acc = history_base.history['val_accuracy'] + history_fine.history['val_accuracy']

    # 2. Concaténation des données de perte (Loss)
    loss = history_base.history['loss'] + history_fine.history['loss']
    val_loss = history_base.history['val_loss'] + history_fine.history['val_loss']

    plt.figure(figsize=(12, 6))

    # --- GRAPHIQUE 1 : PRÉCISION ---
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Entraînement (Train)')
    plt.plot(val_acc, label='Validation (Test)')
    plt.ylim([0, 1]) # On fixe l'échelle entre 0 et 100%
    
    # Ligne verticale pour marquer le début du Fine-Tuning
    plt.plot([initial_epochs-1, initial_epochs-1], plt.ylim(), label='Début Fine-Tuning', linestyle='--', color='green')
    
    plt.legend(loc='lower right')
    plt.title('Précision (Accuracy)')
    plt.xlabel('Époques')

    # --- GRAPHIQUE 2 : PERTE ---
    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Entraînement (Train)')
    plt.plot(val_loss, label='Validation (Test)')
    plt.ylim([0, max(plt.ylim())]) # Echelle automatique pour le max
    
    # Ligne verticale pour marquer le début du Fine-Tuning
    plt.plot([initial_epochs-1, initial_epochs-1], plt.ylim(), label='Début Fine-Tuning', linestyle='--', color='green')
    
    plt.legend(loc='upper right')
    plt.title('Perte (Loss)')
    plt.xlabel('Époques')

    plt.show()

# --- Utilisation ---
# Remplacez '5' par le nombre d'époques de votre PREMIER entraînement
comparer_historiques(history, history_fine, initial_epochs=5)